<a href="https://colab.research.google.com/github/pinecone-io/examples/blob/master/docs/assistant-quickstart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Install the Python SDK and plugin

In [1]:
!pip install pinecone pinecone-plugin-assistant

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.4/421.4 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.2/235.2 kB 16.5 MB/s eta 0:00:00


# 2. Connect to Pinecone

You need an API key to make calls to your assistant.

Use the widget below to generate a key. If you don't have a Pinecone account, the widget will sign you up for the free Starter plan.

In [2]:
# install dependencies

!pip install -qU pinecone-notebooks

from pinecone_notebooks.colab import Authenticate

Authenticate()


Save your API key to use throughout the notebook:

In [3]:
import os
api_key = os.environ.get("PINECONE_API_KEY")

# 3. Create assistant

In [4]:
from pinecone import Pinecone

pc = Pinecone(api_key=api_key)

assistant = pc.assistant.create_assistant(
    assistant_name="quickstart-assistant",
    instructions="Use American English for spelling and grammar.", # Description or directive for the assistant to apply to all responses.
    timeout=30 # Maximum seconds to wait for assistant status to become "Ready" before timing out.
)

# 3. Upload data

For this quickstart, [download a sample 10-k filing file](https://s22.q4cdn.com/959853165/files/doc_financials/2023/ar/Netflix-10-K-01262024.pdf) to your local device. Then, upload it to the notebook's local files.

In [5]:
# Get the assistant

from pinecone import Pinecone

pc = Pinecone(api_key)

assistant = pc.assistant.Assistant(
    assistant_name="quickstart-assistant",
)

# Upload a file.
response = assistant.upload_file(
    file_path="/Netflix-10-K-01262024.pdf",
    metadata={"medium": "filing"},
    timeout=None
)

print("File uploaded.")


File uploaded.


# 4. View files

In [6]:
from pinecone import Pinecone
import pprint

pc = Pinecone(api_key)

assistant = pc.assistant.Assistant(
    assistant_name="quickstart-assistant",
)

# List files in your assistant.
files = assistant.list_files()

pprint.pp(files)

[FileModel(name='Netflix-10-K-01262024.pdf',
           id='f6cedaee-96e4-49c4-bc72-bbd62b62434a',
           metadata={'medium': 'filing'},
           created_on='2025-02-20T18:37:03.338791010Z',
           updated_on='2025-02-20T18:37:48.013250637Z',
           status='Available',
           percent_done=1.0,
           signed_url=None,
           error_message=None,
           size=1073470.0)]


# 5. Chat with assistant

In [8]:
# Chat with the assistant.
from pinecone_plugins.assistant.models.chat import Message

chat_context = [Message(role="user", content="Who is Netflix's CEO and CFO?")]
response = assistant.chat(messages=chat_context)

pprint.pp(response)

ChatResponse(id='000000000000000017c8786a14e6d89d',
             model='gpt-4o-2024-11-20',
             usage=Usage(prompt_tokens=11622,
                         completion_tokens=42,
                         total_tokens=11664),
             message=Message(content="Netflix's Co-Chief Executive Officers "
                                     '(CEOs) are Ted Sarandos and Greg Peters, '
                                     'and the Chief Financial Officer (CFO) is '
                                     'Spencer Neumann.',
                             role='assistant'),
             finish_reason='stop',
             citations=[Citation(position=135,
                                 references=[Reference(file=FileModel(name='Netflix-10-K-01262024.pdf',
                                                                      id='f6cedaee-96e4-49c4-bc72-bbd62b62434a',
                                                                      metadata={'medium': 'filing'},
                        

# 6. Evaluate the answer

Evaluate the correctness and completeness of a response from an assistant. For more information, refer to [Understanding evaluation](https://docs.pinecone.io/guides/assistant/understanding-evaluation).

In [12]:
import requests

answer = response.message.content

eval_data = {
    "question": "Who is Netflix's CEO and CFO?",
    "answer": answer,
    "ground_truth_answer": "Ted Sarandos is co-CEO, Greg Peters is co-CEO, and Spencer Neumann is CFO."
}


headers = {
    "Api-Key": api_key,
    "Content-Type": "application/json"
}

url = "https://prod-1-data.ke.pinecone.io/assistant/evaluation/metrics/alignment"

eval_response = requests.request("POST", url, json=eval_data, headers=headers)

pprint.pp(eval_response.text)

('{"metrics":{"correctness":1.0,"completeness":1.0,"alignment":1.0},"reasoning":{"evaluated_facts":[{"fact":{"content":"Ted '
 'Sarandos is a co-CEO of '
 'Netflix."},"entailment":"entailed"},{"fact":{"content":"Greg Peters is a '
 'co-CEO of Netflix."},"entailment":"entailed"},{"fact":{"content":"Spencer '
 'Neumann is the CFO of '
 'Netflix."},"entailment":"entailed"}]},"usage":{"prompt_tokens":1249,"completion_tokens":55,"total_tokens":1304}}')


# 7. Retrieve context snippets

You can retrieve the context snippets that Pinecone Assistant uses to generate its responses. This data includes relevant chunks, relevancy scores, and references. For more information, refer to [Understanding context snippets](https://docs.pinecone.io/guides/assistant/understanding-context-snippets).

In [13]:
response_snippets = assistant.context(query=chat_context[0].content)

for snippet in response_snippets.snippets:
  pprint.pp(snippet)

Snippet(type='text',
        content='EXHIBIT 31.3\n'
                'CERTIFICATION OF CHIEF FINANCIAL OFFICER\n'
                'PURSUANT TO SECTION 302 OF THE SARBANES-OXLEY ACT OF 2002\n'
                'I, Spencer Neumann, certify that:\n'
                '1. I have reviewed this Annual Report on Form 10-K of '
                'Netflix, Inc.;\n'
                '2. Based on my knowledge, this report does not contain any '
                'untrue statement of a material fact or omit to state a '
                'material fact necessary to make the\n'
                'statements made, in light of the circumstances under which '
                'such statements were made, not misleading with respect to the '
                'period covered by this\n'
                'report;\n'
                '3. Based on my knowledge, the financial statements, and other '
                'financial information included in this report, fairly present '
                'in all material respects the

# 8. Clean up

When you no longer need the quickstart-assistant, delete the it:

In [ ]:
from pinecone import Pinecone
pc = Pinecone(api_key="YOUR_API_KEY")

pc.assistant.delete_assistant(
    assistant_name="example-assistant",
)

# Deleting an assistant also deletes all files uploaded to the assistant.